In [9]:
import pandas as pd
import gcsfs
from google.oauth2.service_account import Credentials
from pyspark.sql import SparkSession
import numpy as np 


In [ ]:
# spark = SparkSession.builder \
#     .appName('energy_prediction') \
#     .getOrCreate()

# spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
# spark._jsc.hadoopConfiguration().set('fs.AbstractFileSystem.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS')
# spark._jsc.hadoopConfiguration().set('google.cloud.auth.service.account.enable', 'true')
# spark._jsc.hadoopConfiguration().set('google.cloud.auth.service.account.json.keyfile', './config/credentials.json')

# df = spark.read.csv('gs://bucket_name/file_path.csv', header=True, inferSchema=True)



In [10]:
# Path to your service account key file
key_file_path = './config/credentials.json'

# Create credentials with the appropriate scope
credentials = Credentials.from_service_account_file(key_file_path, 
    scopes=['https://www.googleapis.com/auth/devstorage.read_write'])

In [11]:
# Create a GCSFileSystem object using these credentials
gcs = gcsfs.GCSFileSystem(token=credentials)

In [17]:
files = gcs.ls('walkingbucket/ashrae-energy-prediction/')


In [18]:
csv_files = [file for file in files if file.endswith('.csv')]

In [19]:
dfs = [pd.read_csv(gcs.open(file)) for file in csv_files]

KeyboardInterrupt: 

In [ ]:
len(dfs)

[]

In [ ]:
# # Then use this GCSFileSystem object when reading the file
# with gcs.open('gs://walkingbucket/ashrae-energy-prediction/') as f:
#     train_df = pd.read_csv(f)
#     weather_train = 
#     weather_test =
#     building_meta =    

In [ ]:
print('Size of train_df data', train.shape)
print('Size of weather_train_df data', weather_train.shape)
print('Size of weather_test_df data', weather_test.shape)
print('Size of building_meta_df data', building_meta.shape)

In [ ]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train.head()
print(train.columns)

In [ ]:
weather_train.head()

In [ ]:
weather_train.columns

In [ ]:
weather_test.head()

In [ ]:
weather_test.columns

In [ ]:
building_meta.head()

In [ ]:
building_meta.columns

## Exploratory Data Analysis